In [15]:
from madden import *
import os

In [66]:
reload(madden)
for pp in range(101):
    print pp, madden.elo_brier_score(pp/100., 1), madden.elo_brier_score(pp/100., 0)

0 -75.0 25.0
1 -73.01 24.99
2 -71.04 24.96
3 -69.09 24.91
4 -67.16 24.84
5 -65.25 24.75
6 -63.36 24.64
7 -61.49 24.51
8 -59.64 24.36
9 -57.81 24.19
10 -56.0 24.0
11 -54.21 23.79
12 -52.44 23.56
13 -50.69 23.31
14 -48.96 23.04
15 -47.25 22.75
16 -45.56 22.44
17 -43.89 22.11
18 -42.24 21.76
19 -40.61 21.39
20 -39.0 21.0
21 -37.41 20.59
22 -35.84 20.16
23 -34.29 19.71
24 -32.76 19.24
25 -31.25 18.75
26 -29.76 18.24
27 -28.29 17.71
28 -26.84 17.16
29 -25.41 16.59
30 -24.0 16.0
31 -22.61 15.39
32 -21.24 14.76
33 -19.89 14.11
34 -18.56 13.44
35 -17.25 12.75
36 -15.96 12.04
37 -14.69 11.31
38 -13.44 10.56
39 -12.21 9.79
40 -11.0 9.0
41 -9.81 8.19
42 -8.64 7.36
43 -7.49 6.51
44 -6.36 5.64
45 -5.25 4.75
46 -4.16 3.84
47 -3.09 2.91
48 -2.04 1.96
49 -1.01 0.99
50 0.0 0.0
51 0.99 -1.01
52 1.96 -2.04
53 2.91 -3.09
54 3.84 -4.16
55 4.75 -5.25
56 5.64 -6.36
57 6.51 -7.49
58 7.36 -8.64
59 8.19 -9.81
60 9.0 -11.0
61 9.79 -12.21
62 10.56 -13.44
63 11.31 -14.69
64 12.04 -15.96
65 12.75 -17.25
66 13.44 -1

In [39]:
MLNFL_ROOT_DIR = os.environ['MLNFL_ROOT']
print(MLNFL_ROOT_DIR)
codeDir = "".join([MLNFL_ROOT_DIR, os.path.sep])

dataRoot = "".join([codeDir, "data", os.path.sep])



/Users/amitbhattacharyya/repos/mlnfl/nfl


In [ ]:
lookupFiles = {'teams' : {'file': 'nflTeams.csv'}, 'seasons': {'file': 'seasons.csv'}, }

lookupDir = "".join([dataRoot, 'lookup', os.path.sep])

logging.info("lookupFiles = %s" % lookupFiles)
logging.info("lookupDir = %s" % lookupDir)

# load reference data
reference_data = ReferenceData(lookupDir)


In [78]:
import pandas

In [79]:
df_all_seasons = pandas.DataFrame()
for season in range(2010,2018):


    # train on previous 3 yrs of data
    testYear = [season]
    trainYears = range(testYear[0]-3,testYear[0])

    # training data set - includes one extra year for prev yr record
    seasons = np.array(trainYears)
    logging.info("training seasons >> {0}".format(seasons))

    # get training data
    # 1 - read all the games
    path_to_lines = dataRoot + "lines/"
    df_all_historical_games = madden.readGamesAll(path_to_lines, seasons)
    # 2 - compile season record for all teams
    df_all_teams = madden.seasonRecord(df_all_historical_games, reference_data)
    # 3 - apply season records and compute other fields for all games
    df_all_historical_games = madden.processGames(df_all_historical_games, df_all_teams, reference_data)
    # 4 - remove extra year of data
    df_all_historical_games = df_all_historical_games[df_all_historical_games.season.isin(seasons)]

    # use different test set
    season_test = np.array(testYear) # should be only one year
    print(season_test)
    logging.info("results for >> {0}".format(season_test))
    # 1 - read all the games
    dfGamesTest = madden.readGamesAll(path_to_lines, season_test)
    # 2 - compile season record for all teams
    dfTeamsTest = madden.seasonRecord(dfGamesTest, reference_data)
    # 3 - apply season records and compute other fields for all games
    dfGamesTest = madden.processGames(dfGamesTest, dfTeamsTest, reference_data)
    # 4 - remove extra year of data
    dfGamesTest = dfGamesTest[dfGamesTest.season.isin(season_test)]

    # get games for testing and predicting -- should be only one year
    season_test = np.array(testYear)

    dfGamesTest = madden.readGamesAll(path_to_lines, season_test)
    dfTeamsTest = madden.seasonRecord(dfGamesTest,reference_data)
    dfGamesTest = madden.processGames(dfGamesTest, dfTeamsTest, reference_data)
    dfGamesTest = dfGamesTest[dfGamesTest.season.isin(season_test)]


    dfTest = dfGamesTest  #[dfGamesTest.gameWeek == week_number]
    lr_classifier = linear_model.LogisticRegression(C=1e5)
    lr_trained_classifier = madden.runScikitClassifier(df_all_historical_games, madden.FEATURE_COLUMNS, lr_classifier)
    df_lr_predict = madden.predictGames(dfTest, lr_trained_classifier, madden.FEATURE_COLUMNS)
    df_all_picks = madden.rankGames(df_lr_predict, reference_data, season_test[0])
    print season, df_all_picks.eloScore.sum()
    df_all_seasons = df_all_seasons.append(df_all_picks)
    


 2018-09-06 11:42:12,948 - INFO - training seasons >> [2007 2008 2009]
 2018-09-06 11:43:05,279 - INFO - results for >> [2010]


[2010]


 2018-09-06 11:43:54,675 - INFO - training data accuracy = 0.717447916667
 2018-09-06 11:43:54,858 - INFO - training seasons >> [2008 2009 2010]


2010 998.007913081


 2018-09-06 11:44:46,258 - INFO - results for >> [2011]


...

2016 1070.33494363


 2018-09-06 11:55:44,869 - INFO - results for >> [2017]


[2017]


 2018-09-06 11:56:36,949 - INFO - training data accuracy = 0.678385416667


2017 1177.33447199


In [82]:
use_cols = ['season','week','Visitor','Home Team','favorite', 'favoredWin', 'predict_proba', 'eloScore']
df_all_seasons[use_cols].to_clipboard()

In [63]:
dfTest.sum()

Date                  2017-09-072017-09-102017-09-102017-09-102017-0...
Visitor               Kansas City ChiefsNew York JetsAtlanta Falcons...
Visitor Score                                                      5242
Home Team             New England PatriotsBuffalo BillsChicago Bears...
Home Score                                                         5878
Line                                                              521.8
Total Line                                                         None
season                                                           516352
week                                                               2325
favorite              New England PatriotsBuffalo BillsAtlanta Falco...
underdog              Kansas City ChiefsNew York JetsChicago BearsBa...
favoredHomeGame                                                     157
divisionGame                                                         96
homeWin                                                         